In [1]:
import os
import time
from transformers import AutoModelForImageClassification, ViTImageProcessor
import torch
from PIL import Image
from sklearn.metrics import accuracy_score
import optimum
from optimum.onnxruntime import ORTModelForImageClassification

In [2]:
path_to_model = "weights/my_model"

extractor = ViTImageProcessor.from_pretrained(path_to_model)
vit_model = AutoModelForImageClassification.from_pretrained(path_to_model)

In [3]:
save_directory = "vit_onnx_optimum"

ort_model = ORTModelForImageClassification.from_pretrained(path_to_model, export=True)
ort_model.save_pretrained(save_directory)
extractor.save_pretrained(save_directory)

Framework not specified. Using pt to export to ONNX.
E:\NEW_MACHINE\Programs\Anaconda\envs\Model_Compression\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Using framework PyTorch: 2.0.1
E:\NEW_MACHINE\Programs\Anaconda\envs\Model_Compression\Lib\site-packages\transformers\models\vit\modeling_vit.py:170: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if num_channels != self.num_channels:
E:\NEW_MACHINE\Programs\Anaconda\envs\Model_Compression\Lib\site-packages\transformers\models\vit\modeling_vit.py:176: TracerWarning: Converting a tensor to a Python boolean might cause the trace to b

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



['vit_onnx_optimum\\preprocessor_config.json']

In [4]:
extractor_onnx = ViTImageProcessor.from_pretrained(save_directory)
vit_model_onnx = ORTModelForImageClassification.from_pretrained(save_directory)

In [5]:
def model_use(model, img):
    with torch.no_grad():
        logits = model(**img).logits

    predicted_label = logits.argmax(-1).item()

    return model.config.id2label[predicted_label]

In [6]:
path = "data/"
images_list = os.listdir(path)

In [7]:
def size_measurement(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()

    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()

    size_all_mb = (param_size + buffer_size) / (1024 ** 2)
    print('model size: {:.3f}MB'.format(size_all_mb))

In [8]:
# Найдем исходный размер модели.
size_measurement(vit_model)

model size: 327.302MB


In [9]:
# Проверим размер ONNX модели.
onnx_file_size = os.path.getsize(save_directory + "/" + "model.onnx")/(1024**2)

print("Размер модели после конвертации в ONNX стал ", onnx_file_size, " мегабайт.")

Размер модели после конвертации в ONNX стал  327.552695274353  мегабайт.


In [10]:
start_time = time.time()

# Собака 1, кошка 0.
target_list = []
predict_list = []

for element in images_list:

    image = Image.open(path + element, mode='r', formats=None)

    inputs = extractor_onnx(image, return_tensors="pt")
    predict = model_use(vit_model_onnx, inputs)
    target = element[:element.find(".")]

    if target == "dog":
        label = 1
    else:
        label = 0

    target_list.append(label)

    if predict == "dogs":
        pr = 1
    else:
        pr = 0

    predict_list.append(pr)

end_time = time.time()

acc = accuracy_score(target_list, predict_list)
print("Точность модели сконвертированной в ONNX = ", acc)
print("Время обработки изображений модели сконвертированной в ONNX = ", end_time-start_time, " секунд")
print("Скорость обработки изображений у модели сконвертированной в ONNX составила  ", len(images_list)/(end_time-start_time), " картинок в секунду")

Точность модели сконвертированной в ONNX =  0.9875
Время обработки изображений модели сконвертированной в ONNX =  17.910914182662964  секунд
Скорость обработки изображений у модели сконвертированной в ONNX составила   8.93310069872779  картинок в секунду
